Assuming that we are given the TIER_1_WHICH_HALF and TIER_2_LETTER_RANGE as inputs

In [148]:
import pandas as pd
import numpy as np
!pip install -q wordcloud
import wordcloud
from nltk.corpus import stopwords
import nltk
import string
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
nltk.download('stopwords')
stop = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [149]:
df = pd.read_csv("data.csv")

In [150]:
# Number of stop words: 
df['stopwords'] = df['text'].apply(lambda x: len([x for x in x.split() if x in stop]))
df[['text','stopwords']].head()

,text,stopwords
0,"What TV comedian worked with White Fang , Blac...",3
1,What country was Mikhail Gorbachev the leader ...,3
2,What company uses the unfamiliar faces of its ...,7
3,Name the Four Horsemen of the Apocalypse .,3
4,How much is a Canadian 1967 twenty dollar gold...,2


In [151]:
# The number of punctuation:
def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return count

df['punctuation'] = df['text'].apply(lambda x: count_punct(x))
df[['text','punctuation']].head()

,text,punctuation
0,"What TV comedian worked with White Fang , Blac...",2
1,What country was Mikhail Gorbachev the leader ...,1
2,What company uses the unfamiliar faces of its ...,1
3,Name the Four Horsemen of the Apocalypse .,1
4,How much is a Canadian 1967 twenty dollar gold...,1


In [152]:
# Number of numerical characters:
df['numerics'] = df['text'].apply(lambda x: len([x for x in x.split() if x.isdigit()]))
df[['text','numerics']].head()

,text,numerics
0,"What TV comedian worked with White Fang , Blac...",0
1,What country was Mikhail Gorbachev the leader ...,0
2,What company uses the unfamiliar faces of its ...,0
3,Name the Four Horsemen of the Apocalypse .,0
4,How much is a Canadian 1967 twenty dollar gold...,1


In [153]:
#Number of Uppercase words:
df['upper'] = df['text'].apply(lambda x: len([x for x in x.split() if x.isupper()]))
df[['text','upper']].head()

,text,upper
0,"What TV comedian worked with White Fang , Blac...",1
1,What country was Mikhail Gorbachev the leader ...,0
2,What company uses the unfamiliar faces of its ...,0
3,Name the Four Horsemen of the Apocalypse .,0
4,How much is a Canadian 1967 twenty dollar gold...,0


In [154]:
# Make all text lower case:
df['text'] = df['text'].apply(lambda x: " ".join(x.lower() for x in x.split()))
df['text'].head()

0    what tv comedian worked with white fang , blac...
1    what country was mikhail gorbachev the leader ...
2    what company uses the unfamiliar faces of its ...
3           name the four horsemen of the apocalypse .
4    how much is a canadian 1967 twenty dollar gold...
Name: text, dtype: object

In [155]:
# Removing Punctuation:
df['text'] = df['text'].str.replace('[^\w\s]','')
df['text'].head()

<ipython-input-155-63436af7d5c6>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['text'] = df['text'].str.replace('[^\w\s]','')


0    what tv comedian worked with white fang  black...
1    what country was mikhail gorbachev the leader of 
2    what company uses the unfamiliar faces of its ...
3            name the four horsemen of the apocalypse 
4    how much is a canadian 1967 twenty dollar gold...
Name: text, dtype: object

In [156]:
# Removal of Stop Words
df['text'] = df['text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
df['text'].sample(10)

41               president ghost said haunt white house
47    nickname frederick holy roman emperor king ger...
37                                         flatfish eat
51                                 aspartame known name
55                    rockefeller sometimes called jdr3
67                            large missouri population
16                                caliente mean english
63                               basque country located
59    find scientific data research papers textile e...
66                layer atmosphere contains ozone layer
Name: text, dtype: object

In [157]:
# Number of Words
df['word_count'] = df['text'].apply(lambda x: len(str(x).split(" ")))
df[['text','word_count']].head()

,text,word_count
0,tv comedian worked white fang black tooth pook...,9
1,country mikhail gorbachev leader,4
2,company uses unfamiliar faces celebrity spokes...,9
3,name four horsemen apocalypse,4
4,much canadian 1967 twenty dollar gold coin worth,8


In [158]:
# Number of characters
df['char_count'] = df['text'].str.len() ## this also includes spaces
df[['text','char_count']].head()

,text,char_count
0,tv comedian worked white fang black tooth pook...,53
1,country mikhail gorbachev leader,32
2,company uses unfamiliar faces celebrity spokes...,72
3,name four horsemen apocalypse,29
4,much canadian 1967 twenty dollar gold coin worth,48


In [159]:
# Average Word Length
def avg_word(sentence):
  words = sentence.split()
  return (sum(len(word) for word in words)/(len(words)+0.000001))
df['avg_word'] = df['text'].apply(lambda x: avg_word(x)).round(1)
df[['text','avg_word']].head()

,text,avg_word
0,tv comedian worked white fang black tooth pook...,5.0
1,country mikhail gorbachev leader,7.2
2,company uses unfamiliar faces celebrity spokes...,7.1
3,name four horsemen apocalypse,6.5
4,much canadian 1967 twenty dollar gold coin worth,5.1


In [160]:
df.head()

,Unnamed: 0,text,TIER_1_WHICH_HALF,TIER_2_LETTER_RANGE,TIER_3_ACTUAL_LETTER,stopwords,punctuation,numerics,upper,word_count,char_count,avg_word
0,0,tv comedian worked white fang black tooth pook...,FIRST 14 LETTERS,BETWEEN H AND N,i,3,2,0,1,9,53,5.0
1,1,country mikhail gorbachev leader,LAST 12 LETTERS,BETWEEN T AND Z,t,3,1,0,0,4,32,7.2
2,2,company uses unfamiliar faces celebrity spokes...,FIRST 14 LETTERS,BETWEEN A AND G,d,7,1,0,0,9,72,7.1
3,3,name four horsemen apocalypse,FIRST 14 LETTERS,BETWEEN H AND N,j,3,1,0,0,4,29,6.5
4,4,much canadian 1967 twenty dollar gold coin worth,FIRST 14 LETTERS,BETWEEN H AND N,h,2,1,1,0,8,48,5.1


In [161]:
df[['TIER_1_WHICH_HALF']] = df[['TIER_1_WHICH_HALF']].apply(LabelEncoder().fit_transform)

In [162]:
df[['TIER_2_LETTER_RANGE']] = df[['TIER_2_LETTER_RANGE']].apply(LabelEncoder().fit_transform)

In [163]:
vectorizer = TfidfVectorizer(analyzer='char')
count_vect_df = pd.DataFrame(vectorizer.fit_transform(df['text']).todense(), columns=vectorizer.get_feature_names_out())
df = pd.concat([df, count_vect_df], axis=1)

In [164]:
df = df.drop(columns=['text','Unnamed: 0'])

In [165]:
y = df['TIER_3_ACTUAL_LETTER']
X = df.drop(columns=['TIER_3_ACTUAL_LETTER'])

In [166]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [167]:
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)

DecisionTreeClassifier()

In [168]:
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           a       1.00      1.00      1.00         1
           b       1.00      0.50      0.67         2
           d       0.00      0.00      0.00         0
           e       0.00      0.00      0.00         0
           g       0.00      0.00      0.00         1
           h       0.00      0.00      0.00         1
           i       0.50      1.00      0.67         1
           k       0.00      0.00      0.00         0
           n       0.00      0.00      0.00         1
           o       0.00      0.00      0.00         1
           p       0.00      0.00      0.00         0
           r       1.00      1.00      1.00         1
           s       1.00      1.00      1.00         1
           u       0.00      0.00      0.00         0
           v       0.00      0.00      0.00         1
           x       0.50      0.50      0.50         2
           y       0.00      0.00      0.00         2
           z       0.00    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.